In [1]:
## 这个notebook聚焦于 Ablation Study，针对 Wikitq 的 Test
from utils.schedule_utils import load_data_split,table_to_str,table_to_str_sql,find_intersection_and_add_row_id,Prepare_Data_for_Operator_Sequence,format_document,batch_rerank_scores,ROLLBACK,merge_clean_and_format_df_dict,retrieve_rows_by_subtables,process_error_analysis_list
import pickle
import numpy as np
import pandas as pd
import os
import json
from FlagEmbedding import FlagReranker
from tqdm import tqdm
from typing import Dict, Any
from utils.evaluator import Evaluator
from utils.prompt_generate import build_wikitq_prompt_from_df,evaluate_predictions,filter_dataframe_from_responses,fix_sql_query,match_subtables,retrieve_rows_by_subtables,build_tab_fact_prompt_from_df
from utils.async_llm import infer_prompts

In [2]:
### 读取必要文件 （step 2）
dataset_name = 'wikitq'
split = 'test'
dataset = load_data_split(dataset_name,split)
wikitq_df_processed = np.load(f'datasets/schedule_test/{dataset_name}/wikitq_df_processed.npy',allow_pickle=True).item()
# wikitq_df_processed = np.load('/data/workspace/yanmy/HybridRAG/H-STAR/datasets/wikitq_df_validation.npy', allow_pickle=True).item()
assert len(dataset) == len(wikitq_df_processed)
wikitq_df = pd.DataFrame(dataset)
long_index = np.load('../datasets/pipeline/valid_data_selection.npy',allow_pickle=True)

In [6]:
result = pd.read_csv('../datasets/pipeline/wikitq/wikitq_test_rewrite_qa_output_4B.csv',index_col=0)
_,_,_ = evaluate_predictions(dataset_name,result.iloc[:100],dataset)

  0%|          | 0/100 [00:00<?, ?it/s]

Correct Samples: 80; Total Samples: 100
Accuracy: 80.00


In [9]:
print(result.iloc[55,-2])

You are an expert on table data.
You must use the table data and the additional evidence to answer the given question.

Procedure:
- Divide the main statement into sub-tasks and answer each sub-task
- Based on the answers, check whether the statement is supported by the table

**NOTE** 
Please be extremely careful, pause, make sure all instructions have been followed and only then output the answer

Response Format:
Begin your response with 'Output: ' and always include the following:
- Decompose: Divide the main question into sub-tasks and answer each sub-task
- Final Answer: Strictly output as a short phrase starting by `therefore, the answer is: "AnswerName1", "AnswerName2"...` form, no other form

- Read the question carefully, understand, and return what the question asks.
- Be careful, make sure you have followed all instructions and only then return the output.
```
<input>
table caption: List of spans
/*
col : tramway | year of inauguration
row 0 : 3s aerial tramway | 2004
row 1

In [4]:
## generate training-free router result
prompt_list = []
for index in range(len(dataset)):
    index = int(index)

    prompt = build_wikitq_prompt_from_df(dataset,wikitq_df_processed[index],index,template_path='../prompts/training_free_router.txt',processed=True)
    prompt_list.append(prompt)
wikitq_df_rewrite_qa_semantic = wikitq_df
wikitq_df_rewrite_qa_semantic['instruction'] = prompt_list

In [6]:
wikitq_df_rewrite_qa_semantic.to_csv('../datasets/ablation/wikitq_test_training_free_router.csv')